## Abstract

In this notebook, we generate the projected population for the year 2043 for the regions of Upper Bavaria (Oberbayern), Lower Bavaria (Niederbayern), and Swabia (Schwaben), using data from the official report:
https://www.statistik.bayern.de/mam/statistik/gebiet_bevoelkerung/demographischer_wandel/demographische_profile/region14.pdf, pages 20–21.


#### Approach:
We first extract the projected population figures for 2043 for each "Gemeinde" as provided in the report. Then, we calculate the share of each "Kommune" within its "Gemeinde" and apply these shares to the 2024 census data to distribute the projected population accordingly.

Next, using the age and sex distributions from the 2024 census, we estimate how the 2043 population is distributed across all age–sex combinations. These proportions are scaled up to match the total 2043 population.

Finally, we integrate city-level data for Munich, which provide more accurate information by age group, as there's a separate report on this. These data were generated separately in the notebook “create_population_2040_munich.ipynb”.

In [1]:
import numpy as np
import pandas as pd
import re

# --- 1) Read the Data ---
census_2024_df = pd.read_csv("census_data_2024.csv", sep=",", dtype={"commune_id": str})  # keep strings as strings
n_distinct = census_2024_df.iloc[:, 0].nunique(dropna=True)
print("Number of distinct comune ids:", n_distinct)
print("In total, the population for Oberbayern, Niederbayern and Schwaben is:", census_2024_df['weight'].sum())
population_2045 = pd.read_csv("pop_oberbayern_niederbayern_schwaben_2043_raw.csv", sep=",", dtype={"kommunen_id_start": str})
processed_pop_munich_2045 = pd.read_csv("pop_munich_2045_processed.csv", dtype={"kommunen_id": str})

Number of distinct comune ids: 1098
In total, the population for Oberbayern, Niederbayern and Schwaben is: 7957065


In [2]:
# Ensure types are as expected
census_2024_df["commune_id"] = census_2024_df["commune_id"].astype(str).str.strip()
census_2024_df["weight"] = pd.to_numeric(census_2024_df["weight"], errors="coerce")

# Drop rows without an ID
census_2024_df = census_2024_df[census_2024_df["commune_id"].notna() & (census_2024_df["commune_id"] != "")]

# Build: commune_id -> total population (sum of weights)
census_2_pop = (
    census_2024_df.groupby("commune_id", dropna=False)["weight"]
             .sum(min_count=1)         # keeps NaN if all weights were NaN for an ID
             .fillna(0)                # optional: treat all-NaN as 0
             .to_dict()
)

In [3]:
# -------------------- Helpers --------------------
def normalize_id(series: pd.Series) -> pd.Series:
    """Return 12-digit string IDs (digits only, left-padded). Drop invalid."""
    s = (series.astype(str)
               .str.strip()
               .str.replace(r"\D", "", regex=True)
               .str.zfill(12))
    return s.where(s.str.fullmatch(r"\d{12}"))

# -------------------- Inputs --------------------
population_2045 = population_2045[["kommunen_id_start","Name_norm","population_2043_abs"]].copy()

try:
    census_2_pop  # dict exists
except NameError:
    # Build from census_df
    census_2024_df = pd.read_csv("census_data_2024.csv", dtype={"commune_id": str})
    census_2024_df["commune_id_norm"] = normalize_id(census_2024_df["commune_id"])
    census_2024_df = census_2024_df[census_2024_df["commune_id_norm"].notna()].copy()
    census_2024_df["weight"] = pd.to_numeric(census_2024_df["weight"], errors="coerce").fillna(0)

    # Sum weights per commune (current population by commune)
    census_2_pop = (census_2024_df.groupby("commune_id_norm")["weight"]
                    .sum(min_count=1).fillna(0).to_dict())

# -------------------- Build commune-level frame --------------------
# Turn census_2_pop dict into a DataFrame
commune_now = (pd.Series(census_2_pop, name="population_2024_kommune")
               .rename_axis("kommunen_id")  # 12-digit id
               .reset_index())

# Derive 5-digit prefix for each commune
commune_now["kommunen_id"] = normalize_id(commune_now["kommunen_id"])
commune_now = commune_now[commune_now["kommunen_id"].notna()].copy()
commune_now["kommunen_id_start"] = commune_now["kommunen_id"].str[:5]

# -------------------- Aggregate current pop by prefix --------------------
grp_now = (commune_now.groupby("kommunen_id_start", dropna=True)["population_2024_kommune"]
           .sum(min_count=1).rename("population_2024_gemeinde")
           .reset_index())

# Attach the group total and the PDF 2043 totals & name (by prefix)
work = (commune_now
        .merge(grp_now, on="kommunen_id_start", how="left")
        .merge(population_2045.rename(columns={"population_2043_abs":"population_2043_gemeinde",
                                               "Name_norm":"name"}),
               on="kommunen_id_start", how="left"))

# -------------------- Compute shares & scaled 2043 --------------------
# Guard against division by zero: if the group total is 0 or NaN, set share=0 and scaled=0
den = work["population_2024_gemeinde"].replace({0: np.nan})
share = work["population_2024_kommune"] / den
work["weight_of_single_kommune_among_gemeinde"] = (share.fillna(0) * 100)  # percent
work["population_2043_gemeinde"] = pd.to_numeric(work["population_2043_gemeinde"], errors="coerce").fillna(0)

work["scaled_population_2043_kommune"] = (
    work["population_2043_gemeinde"] * (work["weight_of_single_kommune_among_gemeinde"] / 100.0)
).round(0).astype(int)

# -------------------- Final columns & save --------------------
result = work[[
    "kommunen_id",                        # 12-digit
    "kommunen_id_start",                  # 5-digit prefix
    "name",                               # from the prefix row
    "population_2024_kommune",            # current pop by commune (from census_2_pop)
    "population_2024_gemeinde",           # sum over the prefix
    "weight_of_single_kommune_among_gemeinde",  # percent
    "population_2043_gemeinde",           # PDF 2043 per prefix
    "scaled_population_2043_kommune"      # allocated 2043 per commune
]].copy()

# Optional: tidy types / rounding
result["weight_of_single_kommune_among_gemeinde"] = result["weight_of_single_kommune_among_gemeinde"].round(6)

In [4]:
result_cleaned = result[["kommunen_id", "population_2024_kommune", "scaled_population_2043_kommune"]]
result_cleaned = result_cleaned.rename(
    columns={
        "population_2024_kommune": "pop_2024",
        "scaled_population_2043_kommune": "pop_2043",
    }
)

In [5]:
# -------------------- Helpers --------------------
def normalize_id(series: pd.Series) -> pd.Series:
    """Return 12-digit string IDs (digits only, left-padded). Drop invalid."""
    s = (series.astype(str)
               .str.strip()
               .str.replace(r"\D", "", regex=True)
               .str.zfill(12))
    return s.where(s.str.fullmatch(r"\d{12}"))

# -------------------- Inputs --------------------
# result_cleaned must have: kommunen_id, pop_2024, pop_2043
# census_df must have: commune_id, sex, age_class, weight
# If you already have them in memory, you can skip these reads.
# result_cleaned = pd.read_csv("result_cleaned.csv", dtype={"kommunen_id": str})
# census_df = pd.read_csv("census_data.csv", dtype={"commune_id": str})

# --- Normalize IDs and clean types
result_cleaned["kommunen_id"] = normalize_id(result_cleaned["kommunen_id"])
result_cleaned = result_cleaned[result_cleaned["kommunen_id"].notna()].copy()

census_2024_df["commune_id_norm"] = normalize_id(census_2024_df["commune_id"])
census_2024_df = census_2024_df[census_2024_df["commune_id_norm"].notna()].copy()
census_2024_df["weight"] = pd.to_numeric(census_2024_df["weight"], errors="coerce").fillna(0)

# -------------------- 1) Enumerate the 26 combinations --------------------
all_combos = (
    census_2024_df[["sex", "age_class"]]
      .drop_duplicates()
      .sort_values(["sex", "age_class"])
      .reset_index(drop=True)
)
# Sanity: we expect 26 distinct combos total
n_combos = len(all_combos)
if n_combos != 26:
    print(f"WARNING: expected 26 (sex, age_class) combos, found {n_combos}. Proceeding with found combos.")

# -------------------- 2) Compute per-(commune_id, combo) weights & fractions --------------------
combo_weights = (
    census_2024_df
      .groupby(["commune_id_norm", "sex", "age_class"], dropna=False)["weight"]
      .sum(min_count=1)
      .reset_index()
)

totals_per_commune = (
    combo_weights
      .groupby("commune_id_norm", dropna=False)["weight"]
      .sum(min_count=1)
      .rename("total_weight_commune")
      .reset_index()
)

combo_weights = combo_weights.merge(totals_per_commune, on="commune_id_norm", how="left")

# Fraction per combo (guard against division by zero)
combo_weights["fraction_2024"] = (
    combo_weights["weight"] / combo_weights["total_weight_commune"].replace({0: np.nan})
).fillna(0.0)

# -------------------- 3) Ensure every commune has all combos (fill missing with 0) --------------------
communes = result_cleaned[["kommunen_id"]].drop_duplicates().rename(columns={"kommunen_id": "commune_id_norm"})
grid = communes.assign(key=1).merge(all_combos.assign(key=1), on="key").drop(columns="key")

grid = grid.merge(
    combo_weights[["commune_id_norm", "sex", "age_class", "fraction_2024"]],
    on=["commune_id_norm", "sex", "age_class"],
    how="left"
)
grid["fraction_2024"] = grid["fraction_2024"].fillna(0.0)

# -------------------- 4) Expand result_cleaned (26 rows per commune) and scale pops --------------------
expanded = (
    grid.merge(
        result_cleaned.rename(columns={"kommunen_id": "commune_id_norm"}),
        on="commune_id_norm",
        how="left"
    )
)

expanded["pop_2024_by_combo"] = expanded["pop_2024"] * expanded["fraction_2024"]
expanded["pop_2043_by_combo"] = expanded["pop_2043"] * expanded["fraction_2024"]

# -------------------- 5) Validations --------------------
# A) Fractions per commune should sum to 1 (or 0 if truly no data and no fallback)
frac_sums = expanded.groupby("commune_id_norm")["fraction_2024"].sum().reset_index()
bad_fracs = frac_sums[~np.isclose(frac_sums["fraction_2024"], 1.0, atol=1e-8) & (frac_sums["fraction_2024"] != 0)]
if not bad_fracs.empty:
    print("WARNING: Some communes have fractions not summing to 1 (and not 0). Examples:")
    print(bad_fracs.head())

# B) Check that sums of distributed pops match originals (within tolerance)
check_2024 = (
    expanded.groupby("commune_id_norm")["pop_2024_by_combo"].sum().reset_index()
    .merge(result_cleaned.rename(columns={"kommunen_id":"commune_id_norm"})[["commune_id_norm","pop_2024"]],
           on="commune_id_norm", how="left")
)
check_2024["diff_2024"] = check_2024["pop_2024_by_combo"] - check_2024["pop_2024"]
if not np.allclose(check_2024["diff_2024"], 0.0, atol=1e-6, rtol=1e-10):
    print("WARNING: pop_2024_by_combo does not perfectly sum back to pop_2024 for some communes.")

check_2043 = (
    expanded.groupby("commune_id_norm")["pop_2043_by_combo"].sum().reset_index()
    .merge(result_cleaned.rename(columns={"kommunen_id":"commune_id_norm"})[["commune_id_norm","pop_2043"]],
           on="commune_id_norm", how="left")
)
check_2043["diff_2043"] = check_2043["pop_2043_by_combo"] - check_2043["pop_2043"]
if not np.allclose(check_2043["diff_2043"], 0.0, atol=1e-6, rtol=1e-10):
    print("WARNING: pop_2043_by_combo does not perfectly sum back to pop_2043 for some communes.")

# -------------------- 6) Final tidy --------------------
result_expanded = expanded[[
    "commune_id_norm",  # == kommunen_id (12 digits)
    "sex",
    "age_class",
    "fraction_2024",
    "pop_2024_by_combo",
    "pop_2043_by_combo",
    "pop_2024",
    "pop_2043",
]].rename(columns={"commune_id_norm": "kommunen_id"})

# (Optional) Round to integers if you need person counts:
# result_expanded["pop_2024_by_combo"] = result_expanded["pop_2024_by_combo"].round(0).astype(int)
# result_expanded["pop_2043_by_combo"] = result_expanded["pop_2043_by_combo"].round(0).astype(int)

# Save if desired
# result_expanded.to_csv("result_expanded_26_combos.csv", index=False)

In [6]:
final_result = result_expanded.rename(
    columns={
        "kommunen_id": "commune_id",
        "pop_2043_by_combo": "weight",
    }
)
final_result=final_result[["commune_id", "sex","age_class","weight"]]

In [7]:
# ---------- Helpers ----------
def normalize_id(s: pd.Series) -> pd.Series:
    return (s.astype(str).str.strip().str.replace(r"\D", "", regex=True).str.zfill(12))

def normalize_sex(s: pd.Series) -> pd.Series:
    # map a variety of spellings to {'female','male'}; everything else -> lowercased as-is
    mapping = {
        "f":"female","w":"female","weiblich":"female","female":"female",
        "m":"male","mann":"male","maennlich":"male","männlich":"male","male":"male"
    }
    return s.astype(str).str.strip().str.lower().map(mapping).fillna(s.astype(str).str.strip().str.lower())

# ---------- 0) Make safe copies ----------
result_final = final_result.copy()
processed_pop_munich_2045 = processed_pop_munich_2045.copy()

# ---------- 1) Normalize keys / types ----------
# result_final
if "kommunen_id" in result_final.columns and "commune_id" not in result_final.columns:
    result_final = result_final.rename(columns={"kommunen_id":"commune_id"})
result_final["commune_id"] = normalize_id(result_final["commune_id"])
result_final["sex"] = normalize_sex(result_final["sex"])
result_final["age_class"] = pd.to_numeric(result_final["age_class"], errors="coerce").astype("Int64")
result_final["weight"] = pd.to_numeric(result_final["weight"], errors="coerce")

# processed_pop_munich_2045
processed_pop_munich_2045["commune_id"] = normalize_id(processed_pop_munich_2045["commune_id"])
processed_pop_munich_2045["age_start"] = pd.to_numeric(processed_pop_munich_2045["age_start"], errors="coerce").astype("Int64")

# ---------- 2) Reshape Munich table to long by sex ----------
updates_long = (
    processed_pop_munich_2045
      .melt(id_vars=["commune_id","age_start"],
            value_vars=["female_2045","male_2045"],
            var_name="sex_var", value_name="weight_new")
      .assign(sex=lambda df: df["sex_var"].str.replace("_2045","",regex=False))
      .rename(columns={"age_start":"age_class"})
      [["commune_id","sex","age_class","weight_new"]]
)

# If there are accidental duplicates per key, collapse them (sum)
updates_long = (updates_long
                .groupby(["commune_id","sex","age_class"], as_index=False)["weight_new"]
                .sum())

# ---------- 3) Check key alignment BEFORE merge ----------
print("Distinct sex in result_final:", sorted(result_final["sex"].dropna().unique())[:10])
print("Distinct sex in updates_long:", sorted(updates_long["sex"].dropna().unique())[:10])

rf_keys = set(zip(result_final["commune_id"], result_final["sex"], result_final["age_class"]))
up_keys = set(zip(updates_long["commune_id"], updates_long["sex"], updates_long["age_class"]))
matches = rf_keys & up_keys
print("Potential key matches:", len(matches), "out of", len(up_keys), "update keys and", len(rf_keys), "result keys")

if len(matches) == 0:
    # Show a few example keys that fail to match
    only_in_updates = list(up_keys - rf_keys)[:10]
    only_in_result  = list(rf_keys - up_keys)[:10]
    print("Examples of update keys not in result_final:", only_in_updates)
    print("Examples of result_final keys not in updates:", only_in_result)

# ---------- 4) Merge & overwrite weight where we have a match ----------
merged = result_final.merge(
    updates_long,
    on=["commune_id","sex","age_class"],
    how="left"
)

# Keep old for reference if you want:
# merged["weight_old"] = merged["weight"]

updated_count = merged["weight_new"].notna().sum()
print("Rows that will be updated:", updated_count)

merged["weight"] = np.where(merged["weight_new"].notna(), merged["weight_new"], merged["weight"])
merged = merged.drop(columns=["weight_new"])

# This is your updated result_final:
result_final = merged
result_final["weight"] = result_final["weight"].astype(int)

# --- Optional: verify Munich totals match the source after update
muc_id = "091620000000"
check = (
    result_final[result_final["commune_id"] == muc_id]
      .groupby("sex")["weight"].sum()
      .rename("sum_in_result_final")
      .to_frame()
      .join(
          updates_long[updates_long["commune_id"] == muc_id]
            .groupby("sex")["weight_new"].sum()
            .rename("sum_from_updates"),
          how="left"
      )
)
print(check)

# final_result_2 = result_final.copy()
# subset = final_result_2[final_result_2["commune_id"] == "091620000000"]

result_final.to_csv("census_data_2043.csv", index=False)

Distinct sex in result_final: ['female', 'male']
Distinct sex in updates_long: ['female', 'male']
Potential key matches: 26 out of 26 update keys and 28548 result keys
Rows that will be updated: 26
        sum_in_result_final  sum_from_updates
sex                                          
female               932273            932273
male                 897248            897248
